In [1]:
%config Completer.use_jedi = False
showOriginalModelString = True
from scipy.optimize import differential_evolution
import numpy as np
import json
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"
plt.style.use('seaborn-deep')
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]
# from MSC_osteogenesis import *
##/ post processing and plotting //##

In [78]:
import numpy as np
import json
import matplotlib.pyplot as plt
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
class Fuzzy_controller:
    def __init__(self,params):
        self.antecedents = {}
        self.consequents = {}
        self.params = params
    def define_antecedents(self):
        pass
    def define_consequents(self):
        #// define consequents
        range_value = np.arange(0, 1, .01)
        early_diff = ctrl.Consequent(range_value, 'early_diff')
        late_diff = ctrl.Consequent(range_value, 'late_diff')
        
        #// define membership functions
        sigma = .05
        diff_intervals = [0,self.params['early_diff_L'],.5,self.params['early_diff_H'], 1]
        early_diff['Z']=fuzz.gaussmf(range_value, diff_intervals[0], sigma)
        early_diff['L']=fuzz.gaussmf(range_value, diff_intervals[1], sigma)
        early_diff['M']=fuzz.gaussmf(range_value, diff_intervals[2], sigma)
        early_diff['H']=fuzz.gaussmf(range_value, diff_intervals[3], sigma)
        early_diff['VH']=fuzz.gaussmf(range_value, diff_intervals[4], sigma)

        diff_intervals = [0,self.params['late_diff_L'],.5,self.params['late_diff_H'], 1]
        late_diff['Z']=fuzz.gaussmf(range_value, diff_intervals[0], sigma)
        late_diff['L']=fuzz.gaussmf(range_value, diff_intervals[1], sigma)
        late_diff['M']=fuzz.gaussmf(range_value, diff_intervals[2], sigma)
        late_diff['H']=fuzz.gaussmf(range_value, diff_intervals[3], sigma)
        late_diff['VH']=fuzz.gaussmf(range_value, diff_intervals[4], sigma)
#         early_diff.view()
        #// Store
        self.consequents['early_diff'] = early_diff
        self.consequents['late_diff'] = late_diff
    def define_rules(self):
        pass
    def forward(self,inputs):
        # print('inputs: {}'.format(inputs))
        for key,value in inputs.items():
            self.controler.input[key] = value
        self.controler.compute()
        # for key,item in self.consequents.items():
        #     item.view(sim=self.controler)
        outputs = self.controler.output
        # print('outputs: {}'.format(outputs))
        return outputs

class Fuzzy_IL8(Fuzzy_controller):
    def __init__(self,params):
        super().__init__(params)
        self.define_antecedents()
        self.define_consequents()
        self.define_rules()
    def define_antecedents(self):
        #// define antecedents
        intervals = [0,self.params['IL8_M'],100]
        IL8 = ctrl.Antecedent(np.arange(intervals[0], intervals[-1], .005), 'IL8')
        # print(IL10_intervals)
        IL8['Neg'] = fuzz.trimf(IL8.universe, [intervals[0], intervals[0],intervals[1]])
        IL8['Med'] = fuzz.trimf(IL8.universe, [intervals[0], intervals[1], intervals[2]])
        IL8['High'] = fuzz.trimf(IL8.universe, [intervals[1], intervals[2], intervals[2]])
        #// store
        self.antecedents['IL8']=IL8

    def define_rules(self):
        #// rules
        IL8 = self.antecedents['IL8']
        early_diff = self.consequents['early_diff']
        late_diff = self.consequents['late_diff']
        early_diff_rules = [
            ctrl.Rule(IL8['High'] , early_diff['VH']),
            ctrl.Rule(IL8['Med'] , early_diff['H']),
            ctrl.Rule(IL8['Neg'] , early_diff['M'])
        ]
        rules = early_diff_rules
        self.controler = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules))
from observations import observations

with open('inferred_params.json') as file:
    inferred_params = json.load(file)
inferred_params['IL1b_d'] = 50
obj = Fuzzy_IL8(inferred_params)
obj.forward(inputs={'IL8':100})

OrderedDict([('early_diff', 0.9532212841848025)])

In [66]:
def scale(x,factor_u, factor_d): 
    """
    Scale the fuzzy controller's output cosidering that the origin is x=0.5
    The scalling is done differently for x > 0.5 than x < 0.5
    x: fuzzy output
    factor: scale factor
    """
    # print('x : {}'.format(x))

    if x >= 0.5:
        factor = factor_u
    else:
        factor = factor_d
    f=(factor-1)*abs(x-0.5)*2+1
    if x >= 0.5:
        return f
    else:
        return 1/f
print(scale(0,100,100))
print(scale(.25,100,100))
print(scale(.49,100,100))
print(scale(.5,100,100))
print(scale(.75,100,100))
print(scale(1,100,100))

0.01
0.019801980198019802
0.3355704697986575
1.0
50.5
100.0
